In [1]:
import math
import numpy as np
from ROOT import gROOT, gDirectory, gPad, gSystem, gStyle
from ROOT import TCanvas, TFile, TProfile, TNtuple, TF1, TH1F, TH2F, THStack, TLegend, TGraphErrors, TMultiGraph, TText, TLatex
from ROOT import TGaxis, TCut
from ROOT import kWhite, kBlack, kGray, kRed, kGreen, kBlue, kYellow, kMagenta, kCyan, kOrange, kSpring, kTeal, kAzure, kViolet, kPink
%jsroot on

Welcome to JupyROOT 6.18/00


In [2]:
gStyle.SetOptFit(0)
gStyle.SetOptStat(0)  
gStyle.SetOptTitle(1)
gStyle.SetTitleBorderSize(0)
gStyle.SetTitleStyle(0)
gStyle.SetMarkerSize(1)
gStyle.SetTitleX(0.2) 
gStyle.SetTitleY(0.9)
gStyle.SetPadLeftMargin(0.2)

In [3]:
# file = TFile.Open("/home/ilc/yokugawa/macros/QQbar250/analysis/default/bbbar_Kgamma35_15161_cuts12_250GeV_eL_btag1_0.8_btag2_0.8_nbins40_noCheat_adrian.root")
# file = TFile.Open("/home/ilc/yokugawa/macros/QQbar250/analysis/kaon_info/bbbar_Kgamma35_15161_cuts12_250GeV_eL_btag1_0.8_btag2_0.8_nbins40_noCheat_adrian.root")
# file = TFile.Open("/home/ilc/yokugawa/macros/QQbar250/analysis/qqbar_2021/dEdx/rootfiles/output_250_all_tracks_ignoreoverlay_angularcorrection_test.root")
file = TFile.Open("/home/ilc/yokugawa/macros/QQbar250/analysis/ssbar/dEdx/rootfiles/AdrianCorr/output_250_all_tracks_ignoreoverlay_angularcorrection_15162.root")


In [4]:
def makePretty(p, color):
#     p.SetStats(0)
    p.SetMarkerColor(color)
    p.SetFillColor(color)
    p.SetMarkerStyle(1)
    p.SetMarkerSize(0.2)
    p.SetTitle("")
    
def makePrettyTGE(tge, color):
    tge.SetMarkerColor(color)
 #   tge.SetMarkerStyle(21)
 #   tge.SetMarkerSize(0.5)
    tge.SetLineColor(color)
    tge.SetTitle("")

In [5]:
def CosMeanGraph(itr, hist, x, y, ex, ey):
    
    for i in range(itr):
        proj  = hist.ProjectionY("proj_kaon",i,i+1)
        x[i]  = hist.GetXaxis().GetBinCenter(i+1)
        y[i]  = proj.GetMean()
        ex[i] = hist.GetXaxis().GetBinWidth(i+1)/2.
        ey[i] = proj.GetRMS()

In [6]:
kaon_dEdx_truth = file.Get("kaon")
proton_dEdx_truth = file.Get("proton")
pion_dEdx_truth = file.Get("pion")
electron_dEdx_truth = file.Get("electron")
muon_dEdx_truth = file.Get("muon")

kaon_dEdx_cos_truth = file.Get("kaon_cos")
proton_dEdx_cos_truth = file.Get("proton_cos")
pion_dEdx_cos_truth = file.Get("pion_cos")
electron_dEdx_cos_truth = file.Get("electron_cos")
muon_dEdx_cos_truth = file.Get("muon_cos")

kaon_dEdx_rad_truth = file.Get("kaon_dEdx_rad_truth")

kaon_dEdx_cos_corr = file.Get("kaon_dEdx_cos_corr")
kaon_dEdx_rad_corr = file.Get("kaon_dEdx_rad_corr")


pdg_selec = file.Get("pdg_selec")

makePretty(kaon_dEdx_truth,kRed)
makePretty(proton_dEdx_truth,kGreen)
makePretty(pion_dEdx_truth,kBlue)
makePretty(electron_dEdx_truth,kBlack)
makePretty(muon_dEdx_truth,kGray)

makePretty(kaon_dEdx_cos_truth,kRed)
makePretty(proton_dEdx_cos_truth,kGreen)
makePretty(pion_dEdx_cos_truth,kBlue)
makePretty(electron_dEdx_cos_truth,kBlack)
makePretty(muon_dEdx_cos_truth,kGray)

makePretty(kaon_dEdx_rad_truth,kRed)

makePretty(kaon_dEdx_cos_corr,kRed)
makePretty(kaon_dEdx_rad_corr,kRed)


AttributeError: 'TObject' object has no attribute 'SetMarkerColor'

### Original Cos

In [ ]:
c1 = TCanvas("c1", "c_corr",0,0,550,500);

In [ ]:
kaon_dEdx_cos_truth.Draw("colz")
c1.Draw()

### dEdx correction

In [ ]:
c4 = TCanvas("c4", "c_corr",0,0,550,500);

In [ ]:
kaon_dEdx_rad_truth.Draw("colz")
c4.Draw()

In [ ]:
kaon_rad = np.empty(200, dtype=float)
kaon_mean_dEdx = np.empty(200, dtype=float)
e_kaon_rad = np.empty(200, dtype=float)
e_kaon_mean_dEdx = np.empty(200, dtype=float)
radbinkaon = kaon_dEdx_rad_truth.GetNbinsX()

CosMeanGraph(radbinkaon, kaon_dEdx_rad_truth, kaon_rad, kaon_mean_dEdx, e_kaon_rad, e_kaon_mean_dEdx)

In [ ]:
kaonproj = TGraphErrors(radbinkaon, kaon_rad, kaon_mean_dEdx, e_kaon_rad, e_kaon_mean_dEdx)

fkaonRad= TF1("fkaonRad","[0]+[1]*TMath::Power(x+[2],[3])",0.199,math.pi/2.0)
fkaonRad.SetLineColor(kRed)

kaonproj.Fit(fkaonRad,"REMS")

In [ ]:
fit = kaonproj.GetFunction("fkaonRad")
p0 = fit.GetParameter(0)
p1 = fit.GetParameter(1)
p2 = fit.GetParameter(2)
p3 = fit.GetParameter(3)
print("fit(x) = %0.3f + %0.3f * (x + %0.3f)^%0.3f" % (p0,p1,p2,p3))

In [ ]:
c4_2 = TCanvas("c4_2", "c_cos_corr",0,0,550,500)
kaonproj.GetXaxis().SetTitle("rad")
kaonproj.GetYaxis().SetTitle("#frac{dE}{dx} [MeV]")
kaonproj.GetYaxis().SetRangeUser(0.1,0.3)

kaonproj.Draw("ALP")
fkaonRad.Draw("same")

In [ ]:
c4_2.Draw()

In [ ]:
c5 = TCanvas("c5", "c_corr",0,0,550,500);

In [ ]:
kaon_dEdx_rad_corr.Draw("colz")
c5.Draw()

In [ ]:
kaon_rad = np.empty(200, dtype=float)
kaon_mean_dEdx = np.empty(200, dtype=float)
e_kaon_rad = np.empty(200, dtype=float)
e_kaon_mean_dEdx = np.empty(200, dtype=float)
radbinkaon = kaon_dEdx_rad_corr.GetNbinsX()

CosMeanGraph(radbinkaon, kaon_dEdx_rad_corr, kaon_rad, kaon_mean_dEdx, e_kaon_rad, e_kaon_mean_dEdx)

kaonproj = TGraphErrors(radbinkaon, kaon_rad, kaon_mean_dEdx, e_kaon_rad, e_kaon_mean_dEdx)

fkaonRad= TF1("fkaonRad","[0]+[1]*TMath::Power(x+[2],[3])",0.199,math.pi/2.0)
fkaonRad.SetLineColor(kRed)

kaonproj.Fit(fkaonRad,"REMS")

In [ ]:
fit = kaonproj.GetFunction("fkaonRad")
p0 = fit.GetParameter(0)
p1 = fit.GetParameter(1)
p2 = fit.GetParameter(2)
p3 = fit.GetParameter(3)
print("fit(x) = %0.3f + %0.3f * (x + %0.3f)^%0.3f" % (p0,p1,p2,p3))

In [ ]:
c4_3 = TCanvas("c4_3", "c_cos_corr",0,0,550,500)
kaonproj.GetXaxis().SetTitle("rad");
kaonproj.GetYaxis().SetTitle("#frac{dE}{dx} [MeV]");
kaonproj.GetYaxis().SetRangeUser(0.1,0.3)

kaonproj.Draw("ALP")
fkaonRad.Draw("same")
c4_3.Draw()

In [ ]:
c4_4 = TCanvas("c4_4", "c_cos_corr",0,0,550,500)

kaon_dEdx_cos_corr.Draw("colz")
c4_4.Draw()